<a href="https://colab.research.google.com/github/ks67426/dw_matrix_road_sign/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hyperopt

In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
import os
import datetime
%load_ext tensorboard
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

In [3]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign/'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim==1: y_train=to_categorical(y_train)
if y_test.ndim==1: y_test=to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy',metrics='accuracy', optimizer='Adam')
  
  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  #path to logs
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(X_train,
            y_train,
            batch_size=params_fit.get('batch_size',128),
            epochs=params_fit.get('epochs',5),
            verbose=params_fit.get('verbose',1),
            validation_data=params_fit.get('validation_data', (X_train, y_train)),
            callbacks=[tensorboard_callback])
  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring = accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)
  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
                  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
                  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
                  MaxPool2D(),
                  Dropout(0.3),
                  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
                  MaxPool2D(), 
                  Dropout(0.3),               
                  Conv2D(filters=64, kernel_size=(3,3), activation='relu',padding='same'),
                  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
                  MaxPool2D(), 
                  Dropout(0.3),  
                  Flatten(),
                  Dense(1024, activation='relu'),
                  Dropout(0.3),
                  Dense(1024, activation='relu'),
                  Dropout(0.3),
                  Dense(num_classes, activation='softmax')
])
  

In [10]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 4s 15ms/step - loss: 2.5750 - accuracy: 0.2833 - val_loss: 1.1404 - val_accuracy: 0.6381
Epoch 2/5
272/272 [==============================] - 4s 13ms/step - loss: 0.8892 - accuracy: 0.7192 - val_loss: 0.2424 - val_accuracy: 0.9243
Epoch 3/5
272/272 [==============================] - 4s 14ms/step - loss: 0.3911 - accuracy: 0.8808 - val_loss: 0.0904 - val_accuracy: 0.9730
Epoch 4/5
272/272 [==============================] - 4s 13ms/step - loss: 0.2368 - accuracy: 0.9298 - val_loss: 0.0450 - val_accuracy: 0.9876
Epoch 5/5
272/272 [==============================] - 4s 13ms/step - loss: 0.1720 - accuracy: 0.9482 - val_loss: 0.0270 - val_accuracy: 0.9925


0.9662131519274376

In [11]:
  model.evaluate(X_test,y_test)

138/138 [==============================] - 0s 3ms/step - loss: 0.1350 - accuracy: 0.9662


[0.13496002554893494, 0.9662131667137146]

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy',metrics='accuracy', optimizer='Adam')
  
  model.fit(X_train,
            y_train,
            batch_size=128, #int(params.get('batch_size',128)),
            epochs=5,
            verbose=0)
  score = model.evaluate(X_test,y_test, verbose=0)
  accuracy=score[1]
  print(params, 'accuracy={}'.format(accuracy))
  return {'loss':-accuracy, 'status': STATUS_OK, 'model':model} #to maximize accuracy

In [0]:
def get_model(params):
  return Sequential([
                  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
                  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
                  MaxPool2D(),
                  Dropout(params['dropout_cnn_block_one']),
                  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
                  MaxPool2D(), 
                  Dropout(params['dropout_cnn_block_two']),               
                  Conv2D(filters=128, kernel_size=(3,3), activation='relu',padding='same'),
                  Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
                  MaxPool2D(), 
                  Dropout((params['dropout_cnn_block_three'])),  
                  Flatten(),
                  Dense(1024, activation='relu'),
                  Dropout((params['dropout_dense_block_one'])),
                  Dense(1024, activation='relu'),
                  Dropout((params['dropout_dense_block_two'])),
                  Dense(num_classes, activation='softmax')
])

In [0]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10), #from 50 to 200 by 20
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one',0.3,0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two',0.3,0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three',0.3,0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one',0.3,0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two',0.3,0.7),
}

In [16]:
best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30, 
    Trials() #where to save
)

{'batch_size': 190.0, 'dropout_cnn_block_one': 0.4108161888603792, 'dropout_cnn_block_three': 0.4542683853702288, 'dropout_cnn_block_two': 0.3679352991183555, 'dropout_dense_block_one': 0.6106289976214193, 'dropout_dense_block_two': 0.5829907281718865}
accuracy=0.9197278618812561
{'batch_size': 110.0, 'dropout_cnn_block_one': 0.3039888925024926, 'dropout_cnn_block_three': 0.33915045322321996, 'dropout_cnn_block_two': 0.40493811164286597, 'dropout_dense_block_one': 0.4857722086557053, 'dropout_dense_block_two': 0.6571197195599122}
accuracy=0.9394558072090149
{'batch_size': 130.0, 'dropout_cnn_block_one': 0.4751540133880393, 'dropout_cnn_block_three': 0.3267090013822323, 'dropout_cnn_block_two': 0.479048766034167, 'dropout_dense_block_one': 0.6732499805813712, 'dropout_dense_block_two': 0.3626038916417975}
accuracy=0.8578231334686279
{'batch_size': 130.0, 'dropout_cnn_block_one': 0.45529339351051995, 'dropout_cnn_block_three': 0.4684551271336517, 'dropout_cnn_block_two': 0.35793900825267

In [0]:
#Three factors influencing minima in SGD